In [3]:
import os
import os.path as osp
import glob

import numpy as np
import cv2
from patchify import patchify
from mobile_sam.utils import transforms

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\user\Documents\slurry_viscnet\MobileSAM_Vortex\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overw

## Define Preprocessing Functions

In [4]:
def apply_morphology(mask, radius_erosion=1, iter_erosion=1, radius_dilation=3, iter_dilation=2, morph_op="opening"):
    """Applies erosion and dilation to the mask."""
    kernel_erosion = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (radius_erosion, radius_erosion))
    kernel_dilation = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (radius_dilation, radius_dilation))

    if morph_op == "opening":
        temp = cv2.erode(mask, kernel_erosion, iterations=iter_erosion)
        output = cv2.dilate(temp, kernel_dilation, iterations=iter_dilation)
    elif morph_op == "closing":
        temp = cv2.dilate(mask, kernel_dilation, iterations=iter_dilation)
        output = cv2.erode(temp, kernel_erosion, iterations=iter_erosion)
    else:
        raise ValueError("Invalid operation. Choose 'opening' or 'closing'.")
    return output

def get_bounding_box(mask):
    """Extracts bounding box from a mask."""
    y_indices, x_indices = np.where(mask > 0)

    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)

    # add perturbation to box coordinates
    H, W = mask.shape
    x_min = max(0, x_min - np.random.randint(0, 20))
    x_max = min(W, x_max + np.random.randint(0, 20))
    y_min = max(0, y_min - np.random.randint(0, 20))
    y_max = min(H, y_max + np.random.randint(0, 20))

    return np.array([x_min, y_min, x_max, y_max])

## Variables

In [5]:
# Data directories
DATA_ROOT = "captures"
IMAGE_SUBDIR = "raw"
MASK_SUBDIR = "masked"

SAVE_ROOT = "processed_data"
IMAGE_SAVE_SUBDIR = "images"
MASK_SAVE_SUBDIR = "masks"
BOX_SAVE_SUBDIR = "boxes"

# Preprocessing parameters
PATCH_SIZE = 256
PATCH_STEP = 256
radius_erosion = 1
iter_erosion = 1
radius_dilation = 3
iter_dilation = 2
original_image_size = (256, 256)
image_encoder_size = 1024

## Preprocess Data

In [ ]:
import os
import shutil
import zipfile
import stat
from pathlib import Path

# File Retrieving Automation

FILE_PATH = "captures"
downloads = os.path.expanduser("~/Downloads")  # downloaded through stirsimul
zip_file_name = FILE_PATH + ".zip"
downloads_zip = os.path.join(downloads, zip_file_name)
current_path = Path(os.getcwd()).resolve() # current directory  # Full path to the original ZIP file
destination_zip_path = os.path.join(current_path, zip_file_name)  # ZIP file directory after moving
extract_to = os.path.join(current_path, FILE_PATH)  # Target folder directory

# Check if ZIP file exists in Downloads
if not os.path.exists(downloads_zip):
    print(f"No ZIP file found: {downloads_zip}")
    exit()

# Move ZIP file to the script's directory
shutil.move(downloads_zip, current_path)
print(f"Moved {zip_file_name} to {current_path}")

# Delete existing"captures folder if it exists
if os.path.exists(extract_to):
    counter = 1
    new_name = extract_to.replace(FILE_PATH, f"{FILE_PATH}_{250218}_{counter:02d}")
    while os.path.exists(new_name):
        counter += 1
        new_name = extract_to.replace(FILE_PATH, f"{FILE_PATH}_{250218}_{counter:02d}")
    os.rename(extract_to, new_name)
    print(f"Modified existing captures folder name: {new_name}")

# Extract the ZIP file
shutil.unpack_archive(destination_zip_path, extract_to)

print(f"Extracted {zip_file_name} to {extract_to}")

# Delete the ZIP file after extraction
os.remove(destination_zip_path)
print(f"Deleted original ZIP file: {destination_zip_path}")

No ZIP file found: C:\Users\user/Downloads\captures.zip


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user/Downloads\\captures.zip'

: 

In [6]:
image_paths = glob.glob(osp.join(DATA_ROOT, IMAGE_SUBDIR, "*.png"))
resizer = transforms.ResizeLongestSide(image_encoder_size)

for image_path in image_paths:
    mask_path = image_path.replace(IMAGE_SUBDIR, MASK_SUBDIR)
    if not osp.exists(mask_path):
        print(f"Mask not found for {image_path}")
        continue
    image = cv2.imread(image_path)[:, :, :3]  # Ensure RGB
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Apply erosion/dilation to the mask
    mask = apply_morphology(mask)

    # Patchify
    img_patches = patchify(image, (PATCH_SIZE, PATCH_SIZE, 3), step=PATCH_STEP) #(numH, numW, 1, patchH, patchW, C)
    mask_patches = patchify(mask, (PATCH_SIZE, PATCH_SIZE), step=PATCH_STEP) #(numH, numW, patchH, patchW)

    patch_cnt = 0
    for i in range(img_patches.shape[0]):
        for j in range(img_patches.shape[1]):
            patch_img = resizer.apply_image(img_patches[i, j, 0, :, :, :])
            patch_mask = mask_patches[i, j, :, :]

            # Skip empty patches
            if np.max(patch_mask) == 0:
                continue

            # Get bounding Box
            box = resizer.apply_boxes(get_bounding_box(patch_mask), original_image_size)

            # Save patch images and masks in 'process_data' as 00001_p01.png (1st image, 1st patch )
            img_basename = int(osp.basename(image_path).replace(".png", "").replace("raw_", ""))
            patch_img_save_path = osp.join(SAVE_ROOT, IMAGE_SAVE_SUBDIR, f"{img_basename:05d}_p{patch_cnt:01d}.png")
            patch_mask_save_path = osp.join(SAVE_ROOT, MASK_SAVE_SUBDIR, f"{img_basename:05d}_p{patch_cnt:01d}.png")
            patch_box_save_path  = osp.join(SAVE_ROOT, BOX_SAVE_SUBDIR, f"{img_basename:05d}_p{patch_cnt:01d}.npy")

            # Ensure directories exist
            os.makedirs(osp.dirname(patch_img_save_path), exist_ok=True)
            os.makedirs(osp.dirname(patch_mask_save_path), exist_ok=True)
            os.makedirs(osp.dirname(patch_box_save_path), exist_ok=True)

            # Save the patches/boxes
            cv2.imwrite(patch_img_save_path, patch_img)
            cv2.imwrite(patch_mask_save_path, patch_mask)
            np.save(patch_box_save_path, box)
      
            patch_cnt += 1

In [7]:
import cv2

img_example = cv2.imread('processed_data/images/00010_p0.png')
box_example = np.load('processed_data/boxes/00010_p0.npy')

print(f'processed image size:{img_example.shape}')
print(f'processed box location:{box_example}')

processed image size:(1024, 1024, 3)
processed box location:[[ 852.  920. 1024. 1024.]]
